In [11]:
import os
from pathlib import Path

import numpy as np
import pandas as pd


from numpy import mean
from numpy import std
import sklearn
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

# Использовать только процессор.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# Изменить уровень отображения логов
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

# Корень проекта.
DIR_ROOT = Path.cwd().parent.parent
# Путь к удаленной директории с ресурсами: данные, модели и т.д.
DIR_REMOTE: Path | None = Path('/home/admin/cafa/resources')

if DIR_REMOTE is not None and DIR_REMOTE.exists():
    DIR_RESOURCE = DIR_REMOTE
else:
    DIR_RESOURCE = DIR_ROOT

In [3]:
print("TensorFlow v" + tf.__version__)

TensorFlow v2.12.1


In [4]:
train_df = pd.read_csv(DIR_RESOURCE / 'data/prepared/train_df.csv')
train_df.head()

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_1015,Column_1016,Column_1017,Column_1018,Column_1019,Column_1020,Column_1021,Column_1022,Column_1023,Column_1024
0,0.049488,-0.032935,0.032473,-0.033723,-0.059505,0.075936,-0.045860,-0.104476,-0.072112,0.038806,...,-0.046516,-0.028773,-0.021878,-0.097883,0.056475,0.080950,-0.020938,-0.043532,0.096463,0.073070
1,-0.044616,0.064925,-0.080263,-0.075338,-0.004731,0.025408,-0.024685,-0.016568,-0.038980,-0.033870,...,-0.040173,0.003925,0.006130,0.007362,-0.066848,0.106882,-0.030134,0.026724,0.027879,-0.048430
2,-0.020128,-0.049779,0.007894,-0.000829,-0.047737,0.086453,-0.038107,-0.036379,0.029611,0.045029,...,0.029388,0.008456,0.000697,0.013502,-0.005968,-0.011571,0.005704,-0.036103,0.007693,0.106234
3,-0.007515,0.060628,0.004045,0.027056,-0.021542,0.010380,-0.025064,-0.055834,0.068238,0.027764,...,0.020792,0.023307,0.009009,0.018211,0.020820,-0.031738,0.013279,-0.018357,0.008087,0.010917
4,0.013468,0.041516,0.018435,-0.035595,0.008770,0.018699,-0.015452,-0.038092,-0.038326,-0.012299,...,-0.044742,-0.025432,-0.060886,-0.026915,0.026342,0.017237,0.014731,-0.016861,-0.016272,0.037054


In [5]:
lables_df = pd.read_csv(
    DIR_RESOURCE / 'data/prepared/train_lbls_top1500_goterms.csv'
)
lables_df.head()

,GO:0005575,GO:0008150,GO:0110165,GO:0003674,GO:0005622,GO:0009987,GO:0043226,GO:0043229,GO:0005488,GO:0043227,...,GO:0034250,GO:0140053,GO:0031345,GO:0098802,GO:0045861,GO:0051783,GO:0031674,GO:0001818,GO:0006874,GO:0016887
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# сокращаем количество белков и меток до 1000, чтобы код выполнялся за минуту
# на этапе тестирования аккуратности модели

lables_df = lables_df.iloc[:1000]
train_df = train_df.iloc[:1000]

# преобразуем в ndarray для работы с кодом evaluate_model()

# arr_lables = lables_df.to_numpy()
# arr_train = train_df.to_numpy()

In [9]:
# собираем модель по baseline с Kaggle
def get_model(n_inputs: int, n_outputs: int):
    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(n_inputs,)),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dense(units=n_outputs, activation='sigmoid'),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
        loss='binary_crossentropy',
        metrics=['binary_accuracy', tf.keras.metrics.AUC()],
    )
    return model

In [ ]:
# посмотреть список метрик sklearn
sklearn.metrics.get_scorer_names()

scoring=['coverage_error', 'label_ranking_average_precision_score']

ошибка ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

Scoring взяты из доки для [Multilabel ranking metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-ranking-metrics)

c scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'] 

ошибка ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [22]:
# конвертируем модель TF в формат доступный sklearn
def get_estimator(X: pd.DataFrame, y: pd.DataFrame):
    estimator = tf.keras.wrappers.scikit_learn.KerasClassifier(
        build_fn=get_model,
        n_inputs=X.shape[1],
        n_outputs=y.shape[1],
        epochs=10,
        batch_size=100,
        verbose=0,
    )
    return estimator


# получаем оценку эффективности модели по кросс валидации с 1 seed
def get_cv_results(seed: int, X: pd.DataFrame, y: pd.DataFrame, scoring: list):
    cv_results = sklearn.model_selection.cross_validate(
        estimator=get_estimator(X, y),
        X=X,
        y=y,
        cv=sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=seed),
        scoring=scoring,
        return_train_score=True,
    )
    return cv_results


# получаем оценку эффективности модели по кросс валидации с разными seeds
def get_cv_results_with_seeds(
    seeds: list, X: pd.DataFrame, y: pd.DataFrame, scoring: list
):
    cv_results_list = []
    for seed in seeds:
        cv_results = get_cv_results(seed, X, y, scoring)
        cv_results_list.append(cv_results)
    return cv_results_list


In [21]:
X: pd.DataFrame = train_df
y: pd.DataFrame = lables_df
seeds = [1, 42]
scoring = ['coverage_error', 'label_ranking_average_precision_score']
cv_results_list = get_cv_results_with_seeds(seeds, X, y, scoring)

cv_results_list

/tmp/ipykernel_499570/1734176933.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = tf.keras.wrappers.scikit_learn.KerasClassifier(


25/25 [==============================] - 0s 4ms/step


/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 221, in accuracy_score
    y_type, y_true, y_pred = 

24/25 [===========================>..] - ETA: 0s

/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 221, in accuracy_score
    y_type, y_true, y_pred = 

...

25/25 [==============================] - 0s 4ms/step


/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 221, in accuracy_score
    y_type, y_true, y_pred = 

25/25 [==============================] - 0s 4ms/step


/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/home/admin/cafa/julia/kaggle_CAFA/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 221, in accuracy_score
    y_type, y_true, y_pred = 

[{'fit_time': array([5.15242648, 4.59871817, 4.31703973, 4.28237867, 4.35139251]),
  'score_time': array([0.32001138, 0.30867362, 0.30026317, 0.30828524, 0.29199409]),
  'test_accuracy': array([nan, nan, nan, nan, nan]),
  'train_accuracy': array([nan, nan, nan, nan, nan]),
  'test_precision_macro': array([nan, nan, nan, nan, nan]),
  'train_precision_macro': array([nan, nan, nan, nan, nan]),
  'test_recall_macro': array([nan, nan, nan, nan, nan]),
  'train_recall_macro': array([nan, nan, nan, nan, nan]),
  'test_f1_macro': array([nan, nan, nan, nan, nan]),
  'train_f1_macro': array([nan, nan, nan, nan, nan])},
 {'fit_time': array([4.47465515, 4.40598297, 4.30589724, 4.31773829, 4.37866926]),
  'score_time': array([0.30471659, 0.3096931 , 0.29823065, 0.2999475 , 0.28750348]),
  'test_accuracy': array([nan, nan, nan, nan, nan]),
  'train_accuracy': array([nan, nan, nan, nan, nan]),
  'test_precision_macro': array([nan, nan, nan, nan, nan]),
  'train_precision_macro': array([nan, nan, na